In [177]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
import pandas as pd
import soundfile as sf
import numpy as np
from sklearn.utils import shuffle
from sys import platform
from sklearn.model_selection import train_test_split

if platform == "linux" or platform == "linux2":
    # linux
    path='/home/vkalbag/projects/'
elif platform == "darwin":
    # OS X
    path='/Users/vedant/Desktop/Programming/'

In [195]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']

d=np.load(f'{path}ScreamDetection/resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'{path}/ScreamDetection/resources/dataset/lookup_new.csv')

EOFError: Ran out of input

In [9]:
df=df.merge(lut[['video_id','band_name']],on='video_id')
df

,video_id,start_time,mid_ts,label,audio,band_name
0,4600fGWcn9o,0.0,0.5,no_vocals,"[0.0, 0.0, 0.0, -3.0517578125e-05, -1.52587890...",Textures
1,4600fGWcn9o,0.5,1.0,no_vocals,"[0.0004730224609375, 0.0001983642578125, -6.10...",Textures
2,4600fGWcn9o,1.0,1.5,no_vocals,"[-4.57763671875e-05, -3.0517578125e-05, 1.5258...",Textures
3,4600fGWcn9o,1.5,2.0,no_vocals,"[3.0517578125e-05, 0.0, -3.0517578125e-05, 0.0...",Textures
4,4600fGWcn9o,2.0,2.5,no_vocals,"[-0.0008087158203125, -0.000885009765625, -0.0...",Textures
...,...,...,...,...,...,...
33815,0m5fIHHfJTw,217.5,218.0,no_vocals,"[1.52587890625e-05, 6.103515625e-05, 0.0001220...",Lamb of God
33816,0m5fIHHfJTw,218.0,218.5,no_vocals,"[6.103515625e-05, 9.1552734375e-05, 0.00012207...",Lamb of God
33817,0m5fIHHfJTw,218.5,219.0,no_vocals,"[0.0, -1.52587890625e-05, -1.52587890625e-05, ...",Lamb of God
33818,0m5fIHHfJTw,219.0,219.5,no_vocals,"[-1.52587890625e-05, -1.52587890625e-05, -1.52...",Lamb of God


In [13]:
feature_df=df[['label','audio','band_name']]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(3,'label_mapped',mapping)

## Undersampling the master data

In [163]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy={0:2462,1:3000,2:3000},random_state=0)
X = feature_df[['audio','band_name']].to_numpy()
y=feature_df['label_mapped'].to_numpy()
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

In [141]:
d=pd.DataFrame()
d['y_under'] = y_under
d['blah'] = 1

print(d.groupby('y_under')['blah'].count())

y_under
0    2462
1    3000
2    3000
Name: blah, dtype: int64


In [169]:
d=pd.DataFrame()
d['band_names'] = band_names
d['blah'] = 1

print(d.groupby('band_names')['blah'].count())

band_names
Abbath                80
After The Burial     104
Amon Amarth          309
Be'lakor             195
Behemoth             398
Born of Osiris        81
Cannibal Corpse       96
Children Of Bodom    282
Children of Bodom    276
Dark Tranquillity    275
Death                186
Decapitated          217
Ensiferum            181
Enslaved             167
Godless               89
Gojira               291
Immortal             109
In Flames            132
Lamb of God          551
Ne Obliviscaris      551
Nevermore            288
Of Mice & Men        447
Opeth                254
Parkway Drive         87
Rings of Saturn       82
Slayer               373
Slipknot             187
Suffocation           85
Suicide Silence      357
Tesseract            385
Textures             511
Thy Art Is Murder     97
Veil of Maya         199
Wintersun            540
Name: blah, dtype: int64


## Creating train-test-validation split

In [179]:
X_train, X_test1, y_train, y_test1 = train_test_split(X_under, y_under,stratify=band_names, test_size=0.3,random_state=42)

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=42)

In [184]:
d=pd.DataFrame()
d['y_train'] = y_train
d['blah'] = 1
print('TRAIN')
print(d.groupby('y_train')['blah'].count())
train = d['blah'].sum()

d=pd.DataFrame()
d['y_test'] = y_test
d['blah'] = 1
print('TEST')
print(d.groupby('y_test')['blah'].count())
test = d['blah'].sum()
d=pd.DataFrame()
d['y_valid'] = y_valid
d['blah'] = 1
print('VALID')
print(d.groupby('y_valid')['blah'].count())
valid = d['blah'].sum()

print(f"Train:Test:Validation - {train}:{test}:{valid}")

TRAIN
y_train
0    1719
1    2094
2    2110
Name: blah, dtype: int64
TEST
y_test
0    351
1    459
2    459
Name: blah, dtype: int64
VALID
y_valid
0    392
1    447
2    431
Name: blah, dtype: int64
Train:Test:Validation - 5923:1269:1270


In [193]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot)
X_test,y_test=shuffle(X_test,y_test_hot)
X_valid,y_valid=shuffle(X_valid,y_valid_hot)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_train-rawaudio.npy', X_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_test-rawaudio.npy', X_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_valid-rawaudio.npy', X_valid)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_train-rawaudio.npy', y_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_test-rawaudio.npy', y_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_valid-rawaudio.npy', y_valid)